- To be used for inference purpose 

    - Inference for momentum.

In [25]:
# Function to get CPU memory usage

def get_cpu_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss  # in bytes

# Function to get GPU memory usage using nvidia-smi
def get_gpu_memory_usage():
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ])
    return int(result.split()[0])  # in MB



In [ ]:
import joblib #Version: 1.2.0
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error #scikit-learn              1.3.0 
import numpy
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 26})

import torch
import torch.nn as nn

from lstm_model_class import LSTMModel  # Import the model class

from lstm_model_class import *



print(torch.cuda.is_available())  # Returns True if a GPU is available
# print(torch.cuda.current_device())  # Returns the index of the current GPU
# print(torch.cuda.get_device_name(0))  # Returns the name of the GPU device

In [ ]:
import os,sys
current_dir = os.getcwd()    # Get the current directory. 
sys.path.append(current_dir) # Append to path 
fn2=os.path.join(current_dir,'..','SourceCode')
sys.path.append(fn2)         # Add the path to the folder containing imports.py

import torch
import torch.nn as nn


import psutil
import os
import subprocess
import imports_kpn as kpn


# Select device : CPU
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model
#loaded_model = LSTMModel(input_size, hidden_size, num_layers, output_size)
#loaded_model.load_state_dict(torch.load('lstm_model_best_pytorch.pth'))

# Load the scripted model
loaded_model = torch.jit.load('scripted_model_lstm_pytorch.pt')

loaded_model.eval()
#model = torch.load('lstm_model_best.pth')

# Move the model to the appropriate device (CPU/GPU)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model.to(device)

# Restrict TensorFlow to only use the GPU
# Function to set CPU as the visible device


In [28]:

 # For ANN, For lstm use imports_kpn
   
        
# import tensorflow as tf  # version 2.6.0



def scale_3D_inference(data_test_features,normalizer_test):
    
    # Step: Reshape Test dataset to 2D
    samples_test, timesteps_test, features_test = data_test_features.shape
    data_test_features = data_test_features.reshape(-1, features_test) 
    
    # Step: Scale features in test dataset         
    norm_test_features=normalizer_test.transform(data_test_features)
        
    #print(normalizer_test.data_max_)
    #print(normalizer_test.data_min_)
    
    # Step: Reshape back to 3D
    norm_test_features = norm_test_features.reshape(samples_test, timesteps_test, features_test)
    
    return norm_test_features


    - Give path locations to test data

In [29]:
momentum=True
pressure=False
allsamples=False

In [30]:
if momentum==True:
    dirFile='../Models/LSTM_HAM'  #ML model for momentum
 
    dirFile_result='../Results'
    # kpn.create_directory_if_not_exists(dirFile_result) 
     
    filename_4_scalingfunction = dirFile + "/Scaling.save"
    
elif pressure==True:
    dirFile='../Models/LSTM_HAM_Pressure'
    # kpn.create_directory_if_not_exists(dirFile)
   
    dirFile_result='../Results_Pressure'
    # kpn.create_directory_if_not_exists(dirFile_result)  
    
    filename_4_scalingfunction = dirFile + "/Scaling.save"
else:
    print('Please choose either momentum or pressure as true')
    


In [ ]:
# Inference
                 
import time
start = time.process_time()
   
    
# Data path for pre-processed data
datapath ="../Data/Coefficient/"

# Data path for modeled data
modelpathbase = "../Data/Coefficient/"

# Data from measurement data
A = kpn.np.load(datapath + 'A.npy') #Features for the momentum
MomentumRes = kpn.np.load(datapath + 'MomentumRes.npy') #label for the momentum

#Split of training and validation data.
xtrain,xtest=kpn.split_timeseries_3D(A,nfrac=0.8)

print(xtrain.shape,xtest.shape)

ytrain,ytest=kpn.split_timeseries_3D(MomentumRes,nfrac=0.8)

if allsamples==True: 
    test_data=xtrain # Load features dataset #[0,:,:].reshape(1,xtrain.shape[1],xtrain.shape[2])
    print(f"Test data shape: , {test_data.shape}")
else:
    test_data=xtrain[0,:,:].reshape(1,xtrain.shape[1],xtrain.shape[2])
    print(f"Test data shape: , {test_data.shape}")
    
print(f"CPU Memory usage before start: {get_cpu_memory_usage() / (1024 ** 2):.2f} MB")
print(f"GPU Memory usage before start: {get_gpu_memory_usage()} MB")
normalizer=joblib.load(filename_4_scalingfunction)

#model.eval()
timely=[]
start2 = time.process_time()
num=1000


#loaded_model.load_state_dict(torch.load('lstm_model_best.pth'))
#loaded_model.eval()

# Run inference
# Convert the outputs to numpy array if needed
#test_outputs = test_outputs.numpy()
#model.eval()

for i in range(num):       

    norm_test_features=scale_3D_inference(test_data,normalizer) # data_test_features 
    # Convert test numpy array to PyTorch tensor
    test_data_tensor = torch.tensor(norm_test_features, dtype=torch.float32)
    #Predict the residual using the model
    with torch.no_grad():
        test_outputs = loaded_model(test_data_tensor)
    

    # print(time.process_time() - start)
    # print(f"Computational time for {num}  no loading, s", time.process_time() - start2)
    
# #Plot the residual

# Check memory usage and time
print(time.process_time() - start2)
print(f"Mean Time: s, {(time.process_time() - start)/num}")
print(f"CPU Memory usage: {get_cpu_memory_usage() / (1024 ** 2):.2f} MB")
print(f"GPU Memory usage: {get_gpu_memory_usage()} MB")


print(norm_test_features.shape)

